<a href="https://colab.research.google.com/github/Sobhan-Gh/Graph-state-functions/blob/main/LC_equivalence.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [156]:
import numpy as np
import networkx as nx
import random

In [81]:
n=int(7)
row=0
G1, G2 = nx.fast_gnp_random_graph(n,0.65), nx.fast_gnp_random_graph(n,0.65)
if nx.is_connected(G1) and nx.is_connected(G2):
    z_1=nx.to_numpy_array(G1).astype(int)
    z_2=nx.to_numpy_array(G2).astype(int)
else:
    raise ValueError('graphs are not connected')

coeff_matrix=np.zeros((n**2,4*n)).astype(int)
for j in range(n):
    for k in range(n):
        for m in range(n):
            row = (n*j+k)
            #a_m
            if m == k: coeff_matrix[row,4*m+0] = z_1[j,k] 
            #b_m
            if m == k and j == k: coeff_matrix[row,4*m+1] = 1
            #c_m
            coeff_matrix[row,4*m+2] = (z_1[m,j] and z_2[m,j])
            #d_m
            if m == j: coeff_matrix[row,4*m+3] = z_2[j,k]

#other constraints (Q matrix = local clifford to be reversible --> determinant == 1)            


In [115]:
#row reduction function
def RowSwap(X,i,j):
  X[[i,j]]=X[[j,i]]
  return X

def Add_Rows(X,i,j):
  X[j]=(X[i]+X[j])%2
  return X
#Returns the row reduced matrix X, the transformed matrix Z and the rank-1
def RowRed(X,Z,pivot):
  n,m=np.shape(X)
  rank=0
  if pivot[1]==(m-1):
    return X,Z,pivot[0]
  else:
    #list of rows with value 1 under the pivot element
    the_ones=[]
    for a in range(pivot[0],n):
      if X[a,pivot[1]]==1:
        the_ones.append(a)
    #check if the column below is empty to skip it
    if not the_ones:
      pivot=[pivot[0],pivot[1]+1]
      X,Z,rank=RowRed(X,Z,pivot)
    else:
      X=RowSwap(X,the_ones[0],pivot[0])
      Z=RowSwap(Z,the_ones[0],pivot[0])
      the_ones.remove(the_ones[0])
      for b in the_ones:
        X=Add_Rows(X,pivot[0],b)
        Z=Add_Rows(Z,pivot[0],b)
      pivot=[pivot[0]+1,pivot[1]+1]
      X,Z,rank=RowRed(X,Z,pivot)
    return X,Z,rank

In [208]:
#find linearly dependent columns in row reduced matrix "x"
#return a list with column indexes
def col_finder(x):
    the_list=[]
    pivot=[0,0]
    for i in range(np.shape(x)[1]-1):
        #print(pivot)
        if x[pivot[0],pivot[1]] == 1:
            pivot=[pivot[0]+1,pivot[1]+1]
        elif x[pivot[0],pivot[1]] == 0:
            the_list.append(pivot[1])
            pivot=[pivot[0],pivot[1]+1]
        else:
          raise ValueError('elements of matrix should be 0 or 1 only')
    if x[-1,-1] == 0: the_list.append(np.shape(x)[1]-1)
    return the_list

In [235]:
#check for rank to see how many independent equations are there = rank of the matrix
rank = np.linalg.matrix_rank(coeff_matrix)
assert (rank < 4*n),"Two graphs/states are not LC equivalent for sure"

pivot=[0,0]
reduced_coeff_matrix,b,c=RowRed(coeff_matrix,coeff_matrix*0,pivot) #row redduction applied 
assert (c==rank or c==rank-1),"row reduced rank is not the same as numpy rank!" 

reduced_coeff_matrix=np.array([i for i in reduced_coeff_matrix if i.any()]) #update the matrix to remove zero rows 
assert (np.shape(reduced_coeff_matrix)[0]==rank),"remaining rows are less than the rank!"

col_list = col_finder(reduced_coeff_matrix) #finding linear dependent columns!

var_vec=np.zeros((4*n,1))
for i in col_list:
    var_vec[i]=np.random.randint(2, size=(1, 1))[0,0]
#var_vec = a random choice of the variables' vector for the n-rank parameters that the equations cannot handle!

#find the matrix equation Ax=b 
#A is the square(rank x rank) matrix out of the reduced_coeff_matrix
#x is the the vector of length=rank to be found by A^(-1)*b
#b is the vector obtained from randomly choosing the extra unknowns of vector rand_vec: b= (-1)* reduced_coeff_matrix * var_vec


A = np.delete(reduced_coeff_matrix,col_list,axis=1) #removing linear dependent columns! 
b = (reduced_coeff_matrix @ var_vec)%2
x = ((np.linalg.inv(A))%2 @ b )%2
print(np.shape(b))

#the full var_vec is now the x vector inserted to the var_vec vector to make all 4*n elements
counter=0
for i in range(4*n):
    if i not in col_list:
        var_vec[i]=x[i-counter][0]
    else:
        print(counter,"i",i)
        counter=+1
        

#now check for constraints on a_i to d_i for elements in full_var 
##TODO


np.set_printoptions(threshold=10000)
#print(reduced_coeff_matrix)

(26, 1)
0 i 22
1 i 27


In [165]:
x=np.array([[3,6]])%2
x.T@x
a=np.random.randint(2, size=(5, 1))
b=np.zeros((3,1))
np.append(b,a,axis=0)
print(reduced_coeff_matrix)

[[1 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 1 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
 [0 0 1 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 1]
 [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 1 0 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 1 0 1 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 1 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 1 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 

In [203]:
test = np.array([[1, 2], [3, 4], [5, 6]])
(test[:,0:2])
-2%2
np.random.randint(2, size=(4*n-rank, 1))
pivot=[0,0]
col_finder(reduced_coeff_matrix)

[22, 27]

In [232]:
test = np.array([[1, 2], [3, 4], [5, 6]])
test[[2]]
np.random.randint(2, size=(1, 1))
for i in x:
  print(i[0])

0.0
0.0
1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
0.0
